In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath(".."))  #TODO: MAKE THE SRC PACKAGE WORK
from src.training.new_optimised_train import train_autoencoder, train_cellfate
from src.evaluation.evaluate import *
from src.training.loss_functions import *
from src.preprocessing.preprocessing_functions import *
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from src.models import Encoder, Decoder, Discriminator, mlp_classifier, complex_mlp_classifier
from src.utils import *
from tensorflow.keras import layers, Sequential
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Load data (just after splitting)
train_images = np.load("../data/images/train_images.npy")
test_images = np.load("../data/images/test_images.npy")
train_labels = np.load("../data/labels/train_labels.npy")
test_labels = np.load("../data/labels/test_labels.npy")
train_tracks = np.load("../data/tracks/train_tracks.npy")
test_tracks = np.load("../data/tracks/test_tracks.npy")

# Load full data

x_train_full = np.load('./data/images/time_norm_train_images.npy')[:,0,:,:]
#y_train_full = np.load('./data/labels/train_labels_augmented4.npy')
#x_test = np.load('./data/images/time_norm_test_images.npy')[:,0,:,:]
#y_test = np.load('./data/labels/test_labels.npy')

In [ ]:
dataset_size = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # will probably have to try with different seeds

# Will use frame 0
def data_size_study(dataset_size, train_images, train_labels, train_tracks, test_images, test_labels, test_tracks, x_train_full, seed=42):

    np.random.seed(seed)

    for size in dataset_size:
        
        # Create new output directory folder with the size 

        output_dir=f"./results/data_labelling_study/split_{size}"
        os.makedirs(output_dir, exist_ok=True)

        less_indexes = np.random.choice(np.arange(len(train_labels)), int(size * len(train_labels)), replace=False)

        # Get less training data
        smaller_x_train_images = train_images[less_indexes]
        smaller_y_train = train_labels[less_indexes]
        smaller_x_train_tracks = train_tracks[less_indexes]

        # Augment image data

        ...

        # Stretch intensities of new images (train and test)

        ...

        # Pick only frame zero
        ...

        # IMAGES: Train autoencoder only
        
        config = {
            'batch_size': 30,
            'epochs': 15,
            'learning_rate': 0.001,
            'seed': 42,
            'latent_dim': 2,
            'GaussianNoise_std': 0.003,
            'lambda_recon': 5,
            'lambda_adv': 1, }

        lambda_autoencoder_results = train_autoencoder(config, x_train_full)
        encoder = lambda_autoencoder_results['encoder']
        decoder = lambda_autoencoder_results['decoder']
        discriminator = lambda_autoencoder_results['discriminator']

        # IMAGES: Train AIcellfate with smaller dataset

        config = {
            'batch_size': 30,
            'epochs': 100,
            'learning_rate': 0.001,
            'seed': 42,
            'latent_dim': 2,
            'GaussianNoise_std': 0.003,
            'lambda_recon': 6,
            'lambda_adv': 4, #4
            'lambda_cov': 0.0001,
            'lambda_contra': 8, #8
        }
 
        # DONT FORGET TO REPLACE x_train and y_train with SMALL DATA
        
        lambda_ae_cov_results = train_cellfate(config, encoder, decoder, discriminator, x_train, y_train, x_test, y_test) #lambda_recon=scaled_autoencoder_results['lambda_recon'], lambda_adv=scaled_autoencoder_results['lambda_adv']
        encoder = lambda_ae_cov_results['encoder']
        decoder = lambda_ae_cov_results['decoder']
        discriminator = lambda_ae_cov_results['discriminator']

        save_model_weights_to_disk(encoder, decoder, discriminator, output_dir=output_dir)
        
        evaluator = Evaluation(output_dir)
        z_imgs = encoder.predict(x_train)
        recon_imgs = decoder.predict(z_imgs)
    
        evaluator.reconstruction_images(x_train, recon_imgs[:,:,:,0], epoch=0)
        
        # Visualise latent space
        evaluator.visualize_latent_space(z_imgs, y_train, epoch=0)

        # Covariance matrix
        cov_matrix = cov_loss_terms(z_imgs)[0]
        evaluator.plot_cov_matrix(cov_matrix, epoch=0)
        


    return smaller_x_train_images, smaller_y_train, smaller_x_train_tracks



#np.save("../results/data_labelling_study/smaller_x_train_images.npy", smaller_x_train_images)